In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif,chi2, mutual_info_classif
import numpy as np
from sklearn.pipeline import Pipeline
import random
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statistics
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn import datasets, metrics, model_selection, svm
from sklearn.metrics import RocCurveDisplay
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score
from scipy import stats
from sklearn.metrics import roc_curve, roc_auc_score


In [ ]:
knn_param_space = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

abc_param_space = {
    'n_estimators': [10, 50],
    'learning_rate': [0.001, 0.01, 0.1],
    'algorithm': ['SAMME', 'SAMME.R']
}

lr_param_space = {
    'penalty': ['l2', 'none'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

rf_param_space = {
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'max_features': ['sqrt', 'log2']}

dt_param_space = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'max_features': ['sqrt', 'log2']}
svc_param_space = {
    'C': [0.001, 0.1, 1],
    'kernel': ['linear', 'sigmoid'],
    'degree': [3, 5],
    'probability':[True]
}
gb_param_space = {
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'max_features': ['sqrt', 'log2']
}

bg_param_space = {
    'n_estimators': [10, 50],
}

mlp_param_space = {
    'learning_rate' : ['constant', 'invscaling', 'adaptive'],
    'max_iter' : [100],
    'activation' :['logistic', 'tanh', 'relu']
}
classifiers = {
    'MLP' : (MLPClassifier() , mlp_param_space),
    'DecisionTreeClassifier' :(DecisionTreeClassifier(), dt_param_space),
    'RandomForestClassifier':(RandomForestClassifier(), rf_param_space),
    'KNeighborsClassifier': (KNeighborsClassifier(), knn_param_space),
    'AdaBoostClassifier': (AdaBoostClassifier(), abc_param_space),
    'LogisticRegression': (LogisticRegression(), lr_param_space),
    'SVC': (SVC(), svc_param_space),
    'Bagging' : (BaggingClassifier(),bg_param_space),
    'GradientBoosting': (GradientBoostingClassifier(),gb_param_space)
    }

In [ ]:
def get_rand_test(dataX,dataY,percent,seed,outcome):

  indics_y_cl1 = dataY[dataY[outcome]==0].index.to_list()

  perc_cl = round(dataX.shape[0]*percent/2)

  random.seed(seed)

  indics_y_rand_cl1 = random.sample(indics_y_cl1, perc_cl)

  indics_y_cl2 = dataY[dataY[outcome]==1].index.to_list()
  indics_y_rand_cl2 = random.sample(indics_y_cl2, perc_cl)

  rand_y_cl1 = dataY.loc[indics_y_rand_cl1]
  rand_y_cl2 = dataY.loc[indics_y_rand_cl2]

  rand_x_cl1 = dataX.loc[indics_y_rand_cl1]
  rand_x_cl2 = dataX.loc[indics_y_rand_cl2]

  test_y = pd.concat([rand_y_cl1,rand_y_cl2],axis=0)
  test_x = pd.concat([rand_x_cl1,rand_x_cl2],axis=0)

  train_x = dataX.drop(indics_y_rand_cl1)
  train_x = train_x.drop(indics_y_rand_cl2)

  train_y = dataY.drop(indics_y_rand_cl1)
  train_y = train_y.drop(indics_y_rand_cl2)

  return test_x, test_y, train_x, train_y


In [ ]:
feature_path = "/content/drive/MyDrive/Parkinson & Multiple labels project/Data/new/ci/"
result_path = "/content/drive/MyDrive/Parkinson & Multiple labels project/Result/"
n_feats = [10,50,100]

def append_row(df, row):
    return pd.concat([
                df,
                pd.DataFrame([row], columns=row.index)]
          ).reset_index(drop=True)
datasets = ['deep_ds']
# seeds = [2666, 524, 552, 380, 2170, 8848, 9738, 153, 8586, 7243, 1202, 3680, 3976, 691, 6386, 2748, 8890, 9340, 2707, 6891]
seeds = [6891]

In [ ]:
for dataset in datasets:
  runs_df = pd.DataFrame()
  path = feature_path+dataset+".xlsx"
  X = pd.read_excel(path , sheet_name='Data' , engine='openpyxl' ,header=0).iloc[:,1:]
  Y = pd.read_excel(path, sheet_name='Output' , engine='openpyxl',header=0).iloc[:,1:].iloc[:,4:]

  imputer = SimpleImputer(missing_values= np.nan , strategy='mean')
  X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

  a = []
  scaler = StandardScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

  a = Y.columns.to_list()
  # a.reverse()
  for outcome in a:
    for name, (clf, param_space) in classifiers.items():
      for n_feat in n_feats:
        internals = []
        externals = []
        externals_auc = []
        YL = pd.DataFrame(Y[outcome])

        for seed in seeds:

          smote = SMOTE(random_state=42)
          X_train, y_train = smote.fit_resample(X, YL)

          X_test, y_test, X_train, y_train = get_rand_test(X_train,y_train,0.2,seed,outcome)

          rfe = RFE(LinearRegression(), n_features_to_select=n_feat)
          X_train = rfe.fit_transform(X_train, y_train)
          X_test = rfe.transform(X_test)

          grid_search = GridSearchCV(clf, param_space, n_jobs=-1, cv=5)
          grid_search.fit(X_train, y_train)

          best_regressor = grid_search.best_estimator_

          cv_results = cross_validate(best_regressor, X_train, y_train, cv=5, scoring='roc_auc', return_train_score = True)

          valSC = statistics.mean(cv_results['test_score'])

          y_test_pred = best_regressor.predict(X_test)
          accuracy_test = balanced_accuracy_score(y_test, y_test_pred)

          try:
            auc_test = roc_auc_score(y_test, best_regressor.predict_proba(X_test)[:,1])
          except:
            auc_test = 'NaN'

          print(str(seed) +' n_feat : '+str(n_feat)+' DS : '+dataset+' OUTCOME : ' +outcome+' , CLF : '+name+' , TRN : '+str(valSC)+' , TST ACC: '+str(auc_test) +' , TST AUC: '+str(accuracy_test) )
        run_new_row = pd.Series({"DS" : dataset ,'n_feat' : n_feat , "OUTCOME" : outcome ,"CLF" : name
                        ,"TRN":valSC
                        ,"TST":accuracy_test,"TST AUC":auc_test})
        runs_df = append_row(runs_df, run_new_row)

runs_df.to_csv(result_path+dataset+"_park-run18.csv")